In [1]:
import numpy as np
import cv2
import tensorflow as tf
import Dataset
import time
import os
import ConfigParser
import sys
import argparse 
import psutil
Dataset=reload(Dataset)

In [2]:
class Hallucinator ():    
    def __init__ (self,config_file,scale,gpu_num):
        print ("Initializing Hallucinator class")
        self.scale=scale
        self.gpu ="/gpu:{}".format(gpu_num)
        self.readConfiguration(config_file)
        self.depth=tf.placeholder(tf.float32,(None,self.imageHeight,self.imageWidth,self.channels),name='depthInput')             
        self.rgb  =tf.placeholder(tf.float32,(None,self.imageHeight,self.imageWidth,self.channels),name='grountTruth')
        self.dataObj = Dataset.dataReader(self.dataArguments)
        if not os.path.exists(self.summary_writer_dir):
            os.makedirs(self.summary_writer_dir)
        if not os.path.exists(self.modelLocation):
            os.makedirs(self.modelLocation)
        logPath = os.path.join(self.logDir,self.modelName)
        if not os.path.exists(logPath):
            os.makedirs(logPath)
        self.logDir = os.path.join(logPath,'log.txt')
        self.logger = open(self.logDir,'w')
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_num)
        self.sess = None
        
    def readConfiguration(self,config_file):
        print ("Reading configuration File")
        config = ConfigParser.ConfigParser()
        config.read(config_file)
        self.imageWidth=int(int(config.get('DATA','imageWidth'))/self.scale)
        self.imageHeight=int(int(config.get('DATA','imageHeight'))/self.scale)
        self.channels=int (config.get('DATA','channels'))
        self.train_file = config.get ('DATA','train_file')
        self.test_file  = config.get ('DATA','test_file')
        self.batchSize  = int(config.get ('DATA','batchSize'))
        self.dataArguments = {"imageWidth":self.imageWidth,"imageHeight":self.imageHeight,\
                              "channels" : self.channels, "batchSize":self.batchSize,\
                              "train_file":self.train_file,"test_file":self.test_file,"scale":self.scale}    
        self.maxEpoch=int(config.get('TRAIN','maxEpoch'))
        self.learningRate = float(config.get('TRAIN','learningRate'))
        
        self.print_freq=int(config.get('LOG','print_freq'))
        self.save_freq = int (config.get('LOG','save_freq'))
        self.val_freq = int (config.get('LOG','val_freq'))
        
        self.modelLocation = config.get('LOG','modelLocation')
        self.modelName = config.get('LOG','modelName') +"_s{}".format(self.scale)
        self.checkPoint =  bool(config.get('LOG','checkpoint'))
        self.restoreModelPath =config.get('LOG','restoreModelPath')
        self.logDir = config.get('LOG','logFile')

        if self.checkPoint:
            print ("Using the latest trained model in check point file")
            self.restoreModelPath = tf.train.latest_checkpoint(self.restoreModelPath)
        self.summary_writer_dir =os.path.join(config.get('LOG','summary_writer_dir') ,self.modelName)     
        
        
    def generateImage(self):  # Inference procedure
        with tf.variable_scope(self.modelName, reuse=tf.AUTO_REUSE):
            #layer 1
            conv1 = tf.layers.conv2d(inputs=self.depth,filters=147,kernel_size=(11,11),\
                                         padding="same",name="conv1",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            conv1 = tf.contrib.layers.instance_norm(inputs=conv1)
            conv1 = tf.nn.selu(conv1,name="conv1_actvn")
            
            #layer 2
            conv2 = tf.layers.conv2d(inputs=conv1,filters=36,kernel_size=(11,11),\
                                         padding="same",name="conv2",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            conv2 = tf.contrib.layers.instance_norm(inputs=conv2)
            conv2 = tf.nn.selu(conv2,name='conv2_actvn')
            
            #layer 3
            conv3 = tf.layers.conv2d(inputs=conv2,filters=36,kernel_size=(11,11),\
                                         padding="same",name="conv3",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            conv3 = tf.contrib.layers.instance_norm(inputs=conv3)
            conv3 = tf.nn.selu(conv3,name='conv3_actvn')
            
            #layer 4
            conv4 =  tf.layers.conv2d(inputs=conv3,filters=36,kernel_size=(11,11),\
                                         padding="same",name="conv4",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            conv4 = tf.contrib.layers.instance_norm(inputs=conv4)
            conv4 = tf.nn.selu(conv4,name="conv4_actvn")
            
            #layer 5
            conv5 = tf.layers.conv2d(inputs=conv4,filters=36,kernel_size=(11,11),\
                                         padding="same",name="conv5",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            conv5 = tf.contrib.layers.instance_norm(inputs=conv5)
            conv5 = tf.nn.selu(conv5,name="conv5_actvn")
            
            #layer 6 
            conv6 = tf.layers.conv2d(inputs=conv5,filters=36,kernel_size=(11,11),\
                                         padding="same",name="conv6",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            conv6 = tf.contrib.layers.instance_norm(inputs=conv6)
            conv6 = tf.nn.selu(conv6,name="conv6_actvn")
            
            #layer 7
            conv7 = tf.layers.conv2d(inputs=conv6,filters=36,kernel_size=(11,11),\
                                         padding="same",name="conv7",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            conv7 = tf.contrib.layers.instance_norm(inputs=conv7)
            conv7 = tf.nn.selu(conv7,name="conv7_actvn")
            
            #layer 8
            conv8 = tf.layers.conv2d(inputs=conv7,filters=36,kernel_size=(11,11),\
                                         padding="same",name="conv8",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            conv8 = tf.contrib.layers.instance_norm(inputs=conv8)
            conv8 = tf.nn.selu(conv8,name="conv8_actvn")
            
            #layer 9 
            conv9 = tf.layers.conv2d(inputs=conv8,filters=36,kernel_size=(11,11),\
                                         padding="same",name="conv9",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            conv9 = tf.contrib.layers.instance_norm(inputs=conv9)
            conv9 = tf.nn.selu(conv9,name="conv9_actvn")
            
            #layer 10
            conv10 = tf.layers.conv2d(inputs=conv9,filters=147,kernel_size=(11,11),\
                                         padding="same",name="conv10",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            conv10 = tf.contrib.layers.instance_norm(inputs=conv10)
            conv10 = tf.nn.selu(conv10,name="conv10_actvn")
            
            #Image generation layer 
            outH = tf.layers.conv2d(inputs=conv10,filters=3,kernel_size=(11,11),\
                                         padding="same",name="output",\
                                         kernel_initializer=tf.truncated_normal_initializer)
            return outH
        
        
    def train(self):      
        #with tf.device(self.gpu):
        self.outH=self.generateImage()
        loss= self.loss()
        optimizer=tf.train.AdamOptimizer(learning_rate=self.learningRate)
        Trainables=optimizer.minimize(loss)
        valid_image_summary =tf.summary.image('test_image_output',self.outH)
        loss_summary = tf.summary.scalar('Loss',loss)
        iters=0
        self.saver = tf.train.Saver()
        config=tf.ConfigProto()
        config.gpu_options.allow_growth=True
        self.sess = tf.Session(config=config)
        train_summary_writer=tf.summary.FileWriter(os.path.join(self.summary_writer_dir,'train'),self.sess.graph)
        test_summary_writer=tf.summary.FileWriter(os.path.join(self.summary_writer_dir,'test'),self.sess.graph)
        self.sess.run(tf.global_variables_initializer())
        process = psutil.Process(os.getpid())
        while not self.dataObj.epoch == self.maxEpoch :
            iters+=1
            inp,gt = self.dataObj.nextTrainBatch()
            loss = self.loss()
            t1=time.time()
            _,lval,t_summaries = self.sess.run([Trainables,loss,loss_summary], feed_dict= {self.depth : inp,self.rgb : gt})
            train_summary_writer.add_summary(t_summaries,iters)
            t2=time.time()         
            
            
            print(process.memory_info().rss/100000.)
            if not iters % self.print_freq:
                info = "Model Hallucinatore_s{} Epoch  {} : Iteration : {}/{} loss value : {:0.4f} Time per batch : {:0.3f}s \n".format(self.scale,self.dataObj.epoch,iters,(self.dataObj.epoch+1)*int(self.dataObj.dataLength/self.dataObj.batchSize),lval,t2-t1)
                print (info)   
                self.logger.write(info)
            if not iters % self.save_freq:
                info="Model Saved at iteration {}\n".format(iters)
                print (info)
                self.logger.write(info)
                self.saveModel(iters)
                
            if not iters % self.val_freq :
                val_inp,val_gt  = self.dataObj.nextTestBatch()
                val_loss,v_summaries,v_img_summaries = self.sess.run([loss,loss_summary,valid_image_summary],feed_dict={self.depth : val_inp,self.rgb : val_gt})
                test_summary_writer.add_summary(v_summaries, iters)
                test_summary_writer.add_summary(v_img_summaries,iters)
                info = "Validation Loss at iteration{} : {}\n".format(iters, val_loss)
                print (info)
                self.logger.write(info)
        print ("Training done ")
        self.logger.close()
    
    def testAll(self):
        self.restoreModel()
        loss=[]
        while not self.dataObj.test_epoch == 1 :
            x,y = self.dataObj.nextTestBatch()
            l = self.sess.run(self.loss(),feed_dict= {self.depth : x, self.rgb :y})
            loss.append(l)
        return np.mean(loss)
        
    def getHallucinatedImages(self,image_list):
        with tf.device(self.gpu):
            self.restoreModel()
            img_processed= self.dataObj.preProcessImages(image_list)
            generator = self.generateImage()
            output = self.sess.run(generator,{self.depth:img_processed})
            return output
    
    def loss(self):
        return tf.reduce_mean(2*tf.nn.l2_loss(self.outH-self.rgb))/(self.imageHeight*self.imageWidth*self.dataObj.batchSize*self.channels)   
    def smoothing_loss(self):
        I_Hgrad = tf.images
    def saveModel(self,iters):
        if not os.path.exists (self.modelLocation):
            os.makedirs(self.modelLocation)
        self.saver.save(self.sess,os.path.join(self.modelLocation,self.modelName),global_step = iters)
        
    def restoreModel(self):
        if not self.sess is None:
            if self.sess._opened :
                self.sess.close()
        sess=tf.Session()
        self.outH=self.generateImage()
        sav=tf.train.Saver()
        sav.restore(sess,self.restoreModelPath)
        self.sess = sess

In [ ]:
if __name__ == '__main__':
    sys.argv=['RGBHallucinator','2','0']
    parser =  argparse.ArgumentParser(description = "Mention the scale and GPU parameters")
    parser.add_argument('scale',type = int ,help = " What scale do you want to train it at ")
    parser.add_argument('gpu',type = int,help = " Which GPU do you want to train it in ")
    args= parser.parse_args()
    scale=args.scale
    gpu  =args.gpu
    tf.reset_default_graph()
    H = Hallucinator("config.ini",scale,gpu)
    H.train()
    H.testAll()

Initializing Hallucinator class
Reading configuration File
Using the latest trained model in check point file
Initializing Data Reader
Train files 108256
Test  files 27064
Initialization Complete
22694.912
23019.76576
23152.2304
23263.92832
23287.64416
Model Hallucinatore_s2 Epoch  0 : Iteration : 5/9021 loss value : 11158.0771 Time per batch : 3.969s 

23332.6592
23089.39776
23398.1952
23511.98208
23515.21792
Model Hallucinatore_s2 Epoch  0 : Iteration : 10/9021 loss value : 10088.1250 Time per batch : 3.949s 

23619.95264
23623.31136
23672.70912
23786.53696
23900.40576
Model Hallucinatore_s2 Epoch  0 : Iteration : 15/9021 loss value : 9232.5039 Time per batch : 4.003s 

23903.55968
23950.9504
23692.32896
24002.10944
24115.93728
Model Hallucinatore_s2 Epoch  0 : Iteration : 20/9021 loss value : 8329.2373 Time per batch : 4.348s 

24129.69984
24243.56864
24246.80448
24286.94528
24390.57408
Model Hallucinatore_s2 Epoch  0 : Iteration : 25/9021 loss value : 7831.5835 Time per batch : 4.0

36809.40032
Model Hallucinatore_s2 Epoch  0 : Iteration : 250/9021 loss value : 639.6640 Time per batch : 4.214s 

36915.28192
36908.4416
36956.9792
37070.97088
37187.05152
Model Hallucinatore_s2 Epoch  0 : Iteration : 255/9021 loss value : 598.3073 Time per batch : 4.249s 

37188.89472
37235.46624
36986.75712
37292.07296
37406.26944
Model Hallucinatore_s2 Epoch  0 : Iteration : 260/9021 loss value : 593.3572 Time per batch : 4.351s 

37409.91488
37514.81344
37518.45888
37578.3424
37694.464
Model Hallucinatore_s2 Epoch  0 : Iteration : 265/9021 loss value : 569.1012 Time per batch : 4.283s 

37695.97952
37806.2848
37533.4912
37837.824
37960.45824
Model Hallucinatore_s2 Epoch  0 : Iteration : 270/9021 loss value : 569.0490 Time per batch : 4.292s 

38064.20992
38067.77344
38122.74176
37858.05824
38166.77376
Model Hallucinatore_s2 Epoch  0 : Iteration : 275/9021 loss value : 578.6964 Time per batch : 4.322s 

38280.92928
38284.41088
38392.87296
38509.64992
38513.29536
Model Hallucinatore

Validation Loss at iteration500 : 182.022644043

51159.94112
51274.1376
51277.57824
51530.21952
51134.8736
Model Hallucinatore_s2 Epoch  0 : Iteration : 505/9021 loss value : 185.4020 Time per batch : 4.638s 

51441.50016
51555.9424
51556.22912
51782.49216
51520.22528
Model Hallucinatore_s2 Epoch  0 : Iteration : 510/9021 loss value : 179.3760 Time per batch : 4.336s 

51698.03264
51950.592
51956.24448
51884.11392
52004.94592
Model Hallucinatore_s2 Epoch  0 : Iteration : 515/9021 loss value : 176.1197 Time per batch : 4.406s 

52001.792
52255.1296
51982.82752
52293.75488
52277.78048
Model Hallucinatore_s2 Epoch  0 : Iteration : 520/9021 loss value : 163.2198 Time per batch : 4.382s 

52387.38944
52506.29632
52388.864
52488.3968
52500.19328
Model Hallucinatore_s2 Epoch  0 : Iteration : 525/9021 loss value : 163.6891 Time per batch : 4.282s 

52724.44928
52461.1584
52775.23968
52767.78496
52886.69184
Model Hallucinatore_s2 Epoch  0 : Iteration : 530/9021 loss value : 168.4157 Time per ba

65273.856
65579.4176
65690.624
65823.37536
65823.49824
Model Hallucinatore_s2 Epoch  0 : Iteration : 760/9021 loss value : 99.3019 Time per batch : 4.367s 

65877.72928
65614.11072
65926.67648
66040.87296
66043.57632
Model Hallucinatore_s2 Epoch  0 : Iteration : 765/9021 loss value : 92.1154 Time per batch : 4.379s 

66144.01024
65877.4016
66180.87424
66291.75296
66402.63168
Model Hallucinatore_s2 Epoch  0 : Iteration : 770/9021 loss value : 81.8068 Time per batch : 4.376s 

66402.87744
66414.01856
66537.92256
66537.96352
66627.54304
Model Hallucinatore_s2 Epoch  0 : Iteration : 775/9021 loss value : 94.1561 Time per batch : 4.973s 

66365.93152
66670.75584
66781.67552
66797.9776
66856.83712
Model Hallucinatore_s2 Epoch  0 : Iteration : 780/9021 loss value : 88.6490 Time per batch : 4.420s 

66972.71296
66980.49536
67110.87104
66837.99552
67150.68416
Model Hallucinatore_s2 Epoch  0 : Iteration : 785/9021 loss value : 88.3526 Time per batch : 5.066s 

67261.68576
67262.01344
67362.16064

79496.76544
79607.72608
79718.1952
79714.63168
79716.84352
Model Hallucinatore_s2 Epoch  0 : Iteration : 1015/9021 loss value : 57.1470 Time per batch : 4.417s 

79823.91296
79822.47936
79910.912
79643.36128
79947.89888
Model Hallucinatore_s2 Epoch  0 : Iteration : 1020/9021 loss value : 57.0370 Time per batch : 4.560s 

80058.61376
80164.70016
80164.82304
80163.55328
80269.27104
Model Hallucinatore_s2 Epoch  0 : Iteration : 1025/9021 loss value : 62.7629 Time per batch : 4.590s 

80269.27104
80372.08064
80096.78848
80403.29216
80510.44352
Model Hallucinatore_s2 Epoch  0 : Iteration : 1030/9021 loss value : 57.1232 Time per batch : 4.719s 

80504.70912
80550.87104
80665.14944
80665.31328
80765.3376
Model Hallucinatore_s2 Epoch  0 : Iteration : 1035/9021 loss value : 62.0412 Time per batch : 4.448s 

80498.97472
80804.90496
80914.8416
80922.50112
81030.47168
Model Hallucinatore_s2 Epoch  0 : Iteration : 1040/9021 loss value : 52.1296 Time per batch : 4.482s 

80771.72736
81075.24096
811

...

636979.85536
637085.45024
637066.4448
637189.81632
637205.83168
Model Hallucinatore_s2 Epoch  1 : Iteration : 11390/18042 loss value : 0.4499 Time per batch : 6.828s 

637211.27936
637308.7232
637278.53568
637283.69664
637393.38752
Model Hallucinatore_s2 Epoch  1 : Iteration : 11395/18042 loss value : 0.3920 Time per batch : 6.848s 

637387.3664
637514.67008
637516.71808
637624.15616
637620.18304
Model Hallucinatore_s2 Epoch  1 : Iteration : 11400/18042 loss value : 0.4174 Time per batch : 6.725s 

637624.81152
637721.14944
637726.1056
637856.68608
637852.91776
Model Hallucinatore_s2 Epoch  1 : Iteration : 11405/18042 loss value : 0.3957 Time per batch : 7.520s 

637854.22848
637984.5632
637981.12256
638098.96448
638102.15936
Model Hallucinatore_s2 Epoch  1 : Iteration : 11410/18042 loss value : 0.4670 Time per batch : 7.038s 

638101.504
638227.37408
638231.71584
638361.47712
638353.44896
Model Hallucinatore_s2 Epoch  1 : Iteration : 11415/18042 loss value : 0.5040 Time per batch : 8.

651286.65088
651290.74688
651296.768
651425.5872
651430.912
Model Hallucinatore_s2 Epoch  1 : Iteration : 11635/18042 loss value : 0.3643 Time per batch : 7.230s 

651567.55456
651567.67744
651698.46272
651704.07424
651706.7776
Model Hallucinatore_s2 Epoch  1 : Iteration : 11640/18042 loss value : 0.3977 Time per batch : 7.202s 

651848.21248
651849.64608
651854.19264
651991.61344
651993.78432
Model Hallucinatore_s2 Epoch  1 : Iteration : 11645/18042 loss value : 0.3936 Time per batch : 7.265s 

652126.49472
652130.63168
652130.75456
652272.4352
652273.62304
Model Hallucinatore_s2 Epoch  1 : Iteration : 11650/18042 loss value : 0.4053 Time per batch : 7.273s 

652277.9648
652410.75712
652416.04096
652546.12992
652547.4816
Model Hallucinatore_s2 Epoch  1 : Iteration : 11655/18042 loss value : 0.5293 Time per batch : 7.286s 

652677.5296
652681.0112
652684.86144
652820.15232
652829.85984
Model Hallucinatore_s2 Epoch  1 : Iteration : 11660/18042 loss value : 0.3682 Time per batch : 6.851s

661227.56096
661228.83072
661135.36
660987.45344
661120.4096
Model Hallucinatore_s2 Epoch  1 : Iteration : 11880/18042 loss value : 0.4052 Time per batch : 7.077s 

661197.25056
661012.8896
661021.2864
661025.71008
661160.3456
Model Hallucinatore_s2 Epoch  1 : Iteration : 11885/18042 loss value : 0.3679 Time per batch : 7.041s 

660994.12992
661132.32896
661133.76256
661142.40512
661261.312
Model Hallucinatore_s2 Epoch  1 : Iteration : 11890/18042 loss value : 0.3557 Time per batch : 7.476s 

661266.06336
661374.11584
661376.86016
661379.8912
661512.47872
Model Hallucinatore_s2 Epoch  1 : Iteration : 11895/18042 loss value : 0.3404 Time per batch : 7.626s 

661514.6496
661463.61344
661467.5456
661403.4432
661412.94592
Model Hallucinatore_s2 Epoch  1 : Iteration : 11900/18042 loss value : 0.4671 Time per batch : 7.530s 

661312.79872
661313.16736
661229.85472
661368.70912
661387.14112
Model Hallucinatore_s2 Epoch  1 : Iteration : 11905/18042 loss value : 0.3526 Time per batch : 6.999s 


654905.87648
654682.15296
654684.32384
654686.16704
654827.97056
Model Hallucinatore_s2 Epoch  1 : Iteration : 12125/18042 loss value : 0.3618 Time per batch : 7.345s 

654756.08576
654709.80096
654757.6832
654628.2496
654610.10432
Model Hallucinatore_s2 Epoch  1 : Iteration : 12130/18042 loss value : 0.4843 Time per batch : 7.379s 

654506.5984
654514.46272
654430.24896
654077.09184
653955.072
Model Hallucinatore_s2 Epoch  1 : Iteration : 12135/18042 loss value : 0.3403 Time per batch : 7.072s 

653955.93216
653960.11008
653679.57504
653820.928
653674.25024
Model Hallucinatore_s2 Epoch  1 : Iteration : 12140/18042 loss value : 0.3626 Time per batch : 7.139s 

653621.0432
653617.84832
653619.65056
653761.536
653380.68992
Model Hallucinatore_s2 Epoch  1 : Iteration : 12145/18042 loss value : 0.3700 Time per batch : 7.150s 

653370.44992
653287.26016
653153.81248
653104.29184
653242.08128
Model Hallucinatore_s2 Epoch  1 : Iteration : 12150/18042 loss value : 0.3243 Time per batch : 7.057

642503.14752
642250.42432
642122.30144
642039.23456
641972.14208
Model Hallucinatore_s2 Epoch  1 : Iteration : 12370/18042 loss value : 0.3341 Time per batch : 7.155s 

642113.69984
642116.608
642116.68992
642079.58016
641938.35008
Model Hallucinatore_s2 Epoch  1 : Iteration : 12375/18042 loss value : 0.3203 Time per batch : 8.874s 

641817.14944
641921.88416
641786.63424
641898.65984
641904.4352
Model Hallucinatore_s2 Epoch  1 : Iteration : 12380/18042 loss value : 0.2928 Time per batch : 7.405s 

641762.59072
641505.44384
641511.09632
641649.54112
641501.63456
Model Hallucinatore_s2 Epoch  1 : Iteration : 12385/18042 loss value : 0.3113 Time per batch : 8.400s 

641367.49056
641291.6736
641318.74816
641319.15776
641189.43744
Model Hallucinatore_s2 Epoch  1 : Iteration : 12390/18042 loss value : 0.2770 Time per batch : 7.601s 

641320.67328
641231.83104
641071.55456
641106.98496
640997.1712
Model Hallucinatore_s2 Epoch  1 : Iteration : 12395/18042 loss value : 0.3372 Time per batch : 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
h=Hallucinator("config.ini",1,0)
inp,gt = h.dataObj.nextTestBatch()

edge= tf.image.sobel_edges(h.rgb)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
out,edge = sess.run([h.rgb,edge],feed_dict = {h.rgb:gt})
sess.close()
plt.imshow(out[0])

In [ ]:
print edge.shape

In [ ]:
a=edge[0]

In [ ]:
y = a[:,:,:,0]

In [ ]:
y.shape

In [ ]:
plt.imshow(y)

In [ ]:
x = a[:,:,:,1]
plt.imshow(x)